In [51]:
import nbformat
from pathlib import Path
from nb_helpers.utils import find_nbs

In [52]:
nbs_paths = find_nbs(Path.cwd())

In [53]:
nb_path = nbs_paths[3]
nb_path

Path('/Users/tcapelle/work/examples/colabs/boosting/Credit_Scorecards_with_XGBoost_and_W&B.ipynb')

In [54]:
def idx_login_cell(nb, delete_line=True):
    for i, cell in enumerate(nb["cells"]):
        if cell["cell_type"] == "code":
            if "login" in cell["source"]:
                if delete_line:
                    cell_content = cell["source"].split("\n")
                    cell_content = [line for line in cell_content if "login" not in line]
                    cell["source"] = "\n".join(cell_content)                  
                return i

In [55]:
nb = nbformat.read(nb_path, 4)

In [56]:
cell = nb["cells"][1]

In [57]:
type(cell)

nbformat.notebooknode.NotebookNode

In [58]:
cell

{'attachments': {},
 'cell_type': 'markdown',
 'metadata': {},
 'source': '<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" /><br>\n\n<!--- @wandbcode{xgboost_vehicle_loan_default} -->\n\n<img src="http://wandb.me/mini-diagram" width="600" alt="Weights & Biases" />'}

In [59]:
login_idx = idx_login_cell(nb)

In [60]:
def insert_login_cell(nb, idx, code="!wandb login"):
    n_cells = len(nb["cells"])
    login_cell = nbformat.v4.new_code_cell(source=code)
    nb["cells"].insert(idx+1, login_cell)
    return nb

In [61]:
nb = insert_login_cell(nb, login_idx)

In [62]:
nb["cells"][login_idx] 

{'cell_type': 'code',
 'execution_count': None,
 'metadata': {},
 'outputs': [],
 'source': "import wandb\n\nWANDB_PROJECT ='vehicle_loan_default'"}

In [63]:
nb["cells"][login_idx+1] 

{'id': 'ff6708eb',
 'cell_type': 'code',
 'metadata': {},
 'execution_count': None,
 'source': '!wandb login',
 'outputs': []}

In [64]:
nb["cells"][login_idx+2] 

{'attachments': {},
 'cell_type': 'markdown',
 'metadata': {},
 'source': '## Vehicle Loan Dataset\n\nWe will be using a simplified version of the [Vehicle Loan Default Prediction dataset](https://www.kaggle.com/sneharshinde/ltfs-av-data) from L&T which has been stored in W&B Artifacts. '}

In [65]:
def remove_properties(nb, props=["id", "idx_", "path_"]):
    for cell in nb["cells"]:
        for prop in props:
            cell.pop(prop, None)
    return nb

def remove_empty_cells(nb):
    nb["cells"] = [cell for cell in nb["cells"] if cell["source"] != ""]
    return nb

In [66]:
nb = remove_properties(nb)
nb = remove_empty_cells(nb)

In [67]:
for nb_path in nbs_paths:
    nb = nbformat.read(nb_path, 4)
    login_idx = idx_login_cell(nb)
    if login_idx is not None:
        nb = insert_login_cell(nb, login_idx)
        # delete path_ attribute
    nb.pop("path_", None)
    nb = remove_properties(nb)
    nb = remove_empty_cells(nb)
    nbformat.validate(nb)
    nbformat.write(nb, nb_path)